In [19]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain import PromptTemplate
import faiss
import streamlit as st
from dotenv import load_dotenv
import os
os

<module 'os' (frozen)>

In [15]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
doc_loader = DirectoryLoader(path="../data/", glob="*.pdf", loader_cls=PyPDFLoader)
documents = doc_loader.load()

In [5]:
# split the documents into chunks 

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                               chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", 
                                   model_kwargs={"device": "cpu"})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\USER\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
vector_store.save_local("vector_store/faiss_db")

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_9316\3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [20]:
llm = ChatOpenAI(model_name="gpt-4", openai_api_key = api_key)

C:\Users\USER\AppData\Local\Temp\ipykernel_9316\2860147426.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", openai_api_key = api_key)


In [22]:
chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}), 
    memory=memory
)

In [23]:
query = "In which century did Tardieu, Simon study symbolism in mentally ill art?"
response = chain.run({"question": query})
print(response)


C:\Users\USER\AppData\Local\Temp\ipykernel_9316\1254460869.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"question": query})


Tardieu and Simon studied symbolism in the artwork of the mentally ill in the late nineteenth century.
